In [12]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [21]:
@tf.function
def layer1(X, W1, B1):
    return tf.nn.relu(tf.matmul(X, W1) + B1)

@tf.function
def layer2(layer1, W2, B2):
    return tf.nn.relu(tf.add(tf.matmul(layer1, W2), B2))

In [22]:
@tf.function
def predict(X, W1, B1, W2, B2):
    return layer2(layer1(X, W1, B1), W2, B2)

In [23]:
@tf.function
def L(y, y_hat):
    loss = y*tf.math.log(y_hat)
    loss1 = -tf.reduce_mean(tf.reduce_sum(loss, axis=1)) #ma tran cong tung dong lai
    return tf.train.AdamOptimizer().minimize(loss1)   

In [24]:
data = pd.read_csv('accent-mfcc-data-1.csv')
X = data.drop('language', axis=1)
y = data['language']
data.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12
count,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000
mean,5.645227,-4.270814,2.635319,7.200207,-5.648733,9.810236,-9.408053,5.117328,-1.229432,-2.362288,2.430833,-3.980415
std,5.105302,3.514350,3.635323,4.310095,4.596180,3.624654,2.484117,2.650608,3.634849,5.042034,3.478467,2.985879
min,-6.067831,-14.972962,-6.186583,-8.844231,-15.656596,-3.528964,-15.365665,-2.873862,-15.510974,-11.429178,-13.664104,-13.724103
25%,2.065603,-6.515764,0.137042,4.700874,-8.417684,8.643563,-11.120860,3.482167,-1.749082,-5.083522,1.197789,-5.941409
50%,4.954206,-4.252512,2.029268,7.804680,-6.786670,10.379330,-9.710399,4.843103,-0.389970,-3.323147,3.169703,-4.374334
75%,9.793648,-1.560250,4.178026,10.129165,-4.463308,11.784360,-7.989370,6.588931,0.779993,-1.506037,4.673452,-2.191658
max,17.749851,3.570765,17.066487,16.178942,7.912809,21.445837,-0.424033,13.846083,4.789989,16.326455,9.166066,5.259430


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [26]:
X_train = tf.constant(X_train, dtype=tf.float64)
y_train = tf.constant(y_train, dtype=tf.float64)
X_test = tf.constant(X_test, dtype=tf.float64)
y_test = tf.constant(y_test, dtype=tf.float64)

input_size = X_train.shape[1]
hidden_units_1 = 128
hidden_units_2 = 64
output_size = len(np.unique(y_train))

# Weights and biases (random initialization)
W1 = tf.Variable(tf.random.normal([input_size, hidden_units_1], dtype=tf.float64))
B1 = tf.Variable(tf.random.normal([hidden_units_1], dtype=tf.float64))

W2 = tf.Variable(tf.random.normal([hidden_units_1, hidden_units_2], dtype=tf.float64))
B2 = tf.Variable(tf.random.normal([hidden_units_2], dtype=tf.float64))

W_out = tf.Variable(tf.random.normal([hidden_units_2, output_size], dtype=tf.float64))
b_out = tf.Variable(tf.random.normal([output_size], dtype=tf.float64))
alpha = 0.001
for it in range(350):
    with tf.GradientTape() as t:
        current_loss = L(y, predict(X, W1, B1, W2, B2))
    print("it", it, current_loss)
    dW1, dB1, dW2, dB2 = t.gradient(current_loss, [W1, B1, W2, B2])
    W1.assign_sub(alpha * dW1)
    B1.assign_sub(alpha * dB1)
    W2.assign_sub(alpha * dW2)
    B2.assign_sub(alpha * dB2)
    if it % 10 == 0:
        print(f'Epoch {it}, Loss: {current_loss.numpy()}')

ValueError: could not convert string to float: 'GE'

In [46]:
y_hat = predict(X, W1, B1, W2, B2)
y_pred_labels = tf.argmax(y_hat, axis=1).numpy()
y_true_labels = tf.argmax(y, axis=1).numpy()
print(y_pred_labels)
print(y_true_labels)
    
accuracy = np.mean(y_pred_labels == y_true_labels)
print(accuracy)

[0 0 0 0]
[0 0 0 0]
1.0
